In [2]:
import pymysql
from functools import partial
import requests
import networkx as nx
import freeman as fm
from config import *
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

def run_db_query(connection, query, args=None):
    with connection.cursor() as cursor:
        print('Executando query:')
        cursor.execute(query, args)
        for result in cursor:
            print(result)

connection = pymysql.connect(
    host=myhost,
    user=myuser,
    password=mypass,
    database='twitch')


db = partial(run_db_query, connection)

In [3]:
def get_games(conn):
    with conn.cursor() as cursor:
        cursor.execute(
            'SELECT * FROM game')
        res = cursor.fetchall()
        if res:
            return res
        else:
            return None
        
def get_links(conn):
    with conn.cursor() as cursor:
        cursor.execute(
            'SELECT * FROM links')
        res = cursor.fetchall()
        if res:
            return res
        else:
            return None

def get_links_from_game(conn,game):
    with conn.cursor() as cursor:
        cursor.execute(
            'SELECT * FROM links WHERE links_game_name = %s',(game))
        res = cursor.fetchall()
        if res:
            return res
        else:
            return None
        
        
# def get_username_from_link(conn,game):
#     with conn.cursor() as cursor:
#         cursor.execute(
#             '''SELECT streamer_display_name FROM streamer
#             INNER JOIN on 
#             WHERE streamer_id = %s''',(game))
#         res = cursor.fetchall()
#         if res:
#             return res
#         else:
#             return None
        
def get_username_from_id(conn,_id):
    with conn.cursor() as cursor:
        cursor.execute(
            'SELECT streamer_display_name FROM streamer WHERE streamer_id = %s',(_id))
        res = cursor.fetchone()
        if res:
            return res
        else:
            return None
        
def get_streamer_id_from_game(conn,game_id):
    with conn.cursor() as cursor:
        cursor.execute(
            'SELECT stream_user_id FROM twitch.stream WHERE stream_game_id = %s GROUP BY stream_user_id ;',(game_id))
        res = cursor.fetchall()
        if res:
            return res
        else:
            return None
def get_total_viewer_count(conn,username):
    with conn.cursor() as cursor:
        cursor.execute("SELECT streamer_view_count FROM twitch.streamer WHERE streamer_display_name = %s",(username))
        res=cursor.fetchone()
        if res:
            return res
        else:
            return None
    
    
        
def get_streams_from_game(conn,time1,time2,game):
    with conn.cursor() as cursor:
        cursor.execute(
            'SELECT * FROM stream WHERE stream_game_id = %s AND stream_request_time BETWEEN %s AND %s order by stream_viewer_count desc',(game, time1, time2))
        res = cursor.fetchall()
        if res:
            return res
        else:
            return None

def get_time(conn,time_1,time_2,game_id):
    #print(time_1,time_2,game_id)
    r = get_streams_from_game(conn,time_1,time_2,game_id)
    #print(r)
    sizes = {}
    if r:
        for node in r:
            sizes[node[2]] = node[6]
    return sizes

def get_progression(conn,start_date,end_date,game_id):
    lista = []
    data = start_date
    while(data < end_date):
        lista.append(get_time(conn,str(data),str(data+timedelta(minutes=10)),game_id))
        data += timedelta(minutes=30)
    return lista

In [86]:
time1 = '2019-11-15 13:00:00'
time2 = '2019-11-15 13:10:00'
time3 = '2019-11-15 13:30:00'
time4 = '2019-11-15 13:40:00'


game = '21779' # LOL

def list_streamer_time(time1,time2,game):
    totalview=0
    streams_lol = get_streams_from_game(connection,time1,time2,game)
    Slist=[]
    for a in streams_lol:
        totalview=totalview+a[6]
        Slist.append((a[2],a[6],a[1],a[8])) # user name, viewer count, user id, request time
    return Slist,totalview
streams1=(list_streamer_time(time1,time2,game))
streams2=(list_streamer_time(time3,time4,game))


In [87]:
#print(streams1[0:30])
#print(streams2)

def filter_value( someList, value ):
    for x, y,z,w in someList:
        if x == value :
            yield x,y
            

def get_leavers(list1,list2):
    leavers=[]
    for i in list1[0:30]:
        result=list( filter_value(list2, i[0]))
        if result==[]:
            leavers.append([i[0],i[2]])
    return leavers
        
get_leavers(streams1[0],streams2[0])



[['SoloRenektonOnly', 30227322]]

In [6]:
import pandas as pd 
time1 = datetime(2019, 11, 11, 0, 0)
time2 = datetime(2019, 11, 11, 23, 0)


def get_links_from_to(conn,from_id,to_id):
    with conn.cursor() as cursor:
        cursor.execute(
            'SELECT * FROM links WHERE from_id = %s AND to_id = %s',(from_id, to_id))
        res = cursor.fetchall()
        if res:
            return res
        else:
            return None


In [28]:
def links_dict(conn,game_name):
    with conn.cursor() as cursor:
        cursor.execute(
            '''
            SELECT st1.streamer_display_name, st2.streamer_display_name FROM links
            INNER JOIN streamer st1 ON links.from_id = st1.streamer_id
            INNER JOIN streamer st2 ON links.to_id = st2.streamer_id
            WHERE links_game_name LIKE %s LIMIT 8000
            ''',(game_name))
        res = cursor.fetchall()
        if res:
            return res
        else:
            return None
        
dict_lol = links_dict(connection, "League of Legends")
print(dict_lol)
print(len(dict_lol))

(('Faker', '매드라이프_'), ('Multiplayer', 'KayPeaLoL'), ('Multiplayer', 'wtcN'), ('Multiplayer', 'AlperBicen'), ('Dekar173', 'imaqtpie'), ('Dekar173', 'IWillDominate'), ('Dekar173', 'imls'), ('Dekar173', 'Shiphtur'), ('Dekar173', 'Wingsofdeath'), ('Dekar173', 'SoloRenektonOnly'), ('Dekar173', 'ESL_LOL'), ('Dekar173', 'wickd'), ('Dekar173', 'boxbox'), ('Dekar173', 'NickiTaylor'), ('Dekar173', '1Adrianaries1'), ('Dekar173', 'Akaadian'), ('Dekar173', 'Rakin'), ('Dekar173', 'HanjaroLoL'), ('Dekar173', 'loltyler1'), ('Dekar173', 'JenaUrf'), ('Dekar173', 'Elite500'), ('Dekar173', 'Viper'), ('Dekar173', 'RATIRL'), ('Dekar173', 'luxxbunny'), ('Dekar173', 'TFBlade'), ('Dekar173', 'ipav999'), ('Dekar173', 'PinkWardlol'), ('Dekar173', 'MaryMaybe'), ('Dekar173', 'HeisendongNA'), ('Dekar173', 'STPeach'), ('Dekar173', 'TrackingThePros'), ('Dekar173', 'Barbaholic'), ('Dekar173', 'ChaseShaco'), ('Dekar173', 'Sanchovies'), ('Dekar173', 'KarasMai'), ('Dekar173', 'Yassuo'), ('Dekar173', 'ForestWithin'), ('De

In [43]:
v = [(x,y) for x,y in dict_lol if x =='Voyboy' and y == 'asas']
print(v)

[]


In [82]:
def segue(name1,name2,dicti):
    a = [(x,y) for x,y in dicti if x ==name1 and y == name2]
    if a != []:
        return True
    
    return False

print(segue("Voyboy","loltyler1",dict_lol))
def generate_dataframe(Antes,Depois,dicti):
    # Antes eh [0] lista de sizes e [1] eh o total view de todo mundo somado
    # Depois eh [0] lista de sizes e [1] eh o total view de todo mundo somado
    
    df=[]
    gt = fm.Graph(nx.DiGraph()) # Grafo
    leavers=get_leavers(Antes[0],Depois[0]) # Lista de sizes
    for node in Antes[0]:
        #print(node[0])
        for node2 in Antes[0]:
            #print(segue(node[0],node2[0],dicti))
            if segue(node[0],node2[0],dicti):
                #print("benis")
                gt.add_edges_from([(node[0], node2[0])])
    #return gt
        
    
    if leavers !=[]:
        print('SAIU')
        print("Leavers",leavers)
        for i in Depois[0]:
            a=[tup for tup in Antes[0] if tup[0] == i[0]]  # Checa se o cara ta no Depois
            if a!=[]:
                
                #print(leavers)
                #print(a[0])
                data=[]
                #print(a,i)
                data.append(i[0])
                minDist = None
                
                # Calcula percentagem delta
                percentAntes= (a[0][1]/Antes[1])*100 # a01 eh view da stream
                #print("Porcentagem antes ",percentAntes)
                percentDepois= (i[1]/Depois[1])*100 # i1 view dps
                #print("Porcentagem depois ",percentDepois)
                dif=(percentDepois-percentAntes)
                data.append(dif)
                
                # Pega os viwers totais
                TotalViewer=get_total_viewer_count(connection,i[0])[0]
                data.append(TotalViewer)
                
                # Checa se os leavers seguiam ele
                count_leaver_that_follow = 0
                for leaver in leavers:
                    #print(leaver[1],a[0][2])
                    #links = get_links_from_to(connection, leaver[1], a[0][2]) # Checa se o leaver segue o user
                    links = segue(leaver[0],a[0][0],dicti)
                    if (links):
                        print("Usuario "+leaver[0]+" saiu. E seguia o "+a[0][0])
                        count_leaver_that_follow+=1
                    try:
                        dist = nx.shortest_path_length(gt.reverse(copy=False),source=leaver[0],target=a[0][0])
                        #print(dist)
                        if minDist is None or dist < minDist:
                            minDist = dist
                    except:
                        #print("N tem path")
                        pass
                    

                ## Distancia
                #print(leaver[],a[0][2])
                #print(nx.shortest_path_length(gt,source=,target=a[0][2]))
                        
                data.append(count_leaver_that_follow)
                data.append(i[3])
                data.append(minDist)
                df.append(data)
            #break
    return df, gt


                
data, gt=generate_dataframe(streams1,streams2,dict_lol)
gt.move('kamada_kawai')
gt.label_nodes()
gt.set_all_nodes('labpos','hover')
gt.draw()

True
SAIU
Leavers [['Faker', 43691], ['괴물쥐123', 148057505], ['RATIRL', 57292293], ['테디이', 152714599], ['lol_Khan', 146647519], ['clid', 146466793]]
Usuario 괴물쥐123 saiu. E seguia o 한동숙
Usuario 괴물쥐123 saiu. E seguia o 과로사1


In [59]:
data

[['한동숙',
  8.920359441592261,
  50998396,
  1,
  datetime.datetime(2019, 11, 13, 13, 30, 2),
  None],
 ['超負荷',
  1.9268905222263726,
  10773880,
  0,
  datetime.datetime(2019, 11, 13, 13, 30, 2),
  None],
 ['接接',
  2.840147286458806,
  103055955,
  0,
  datetime.datetime(2019, 11, 13, 13, 30, 3),
  None],
 ['NoWay4u_Sir',
  3.0080324543020946,
  38866827,
  0,
  datetime.datetime(2019, 11, 13, 13, 30, 3),
  None],
 ['Solary',
  1.0876889998246773,
  87110241,
  0,
  datetime.datetime(2019, 11, 13, 13, 30, 3),
  None],
 ['Rakin',
  1.8653326719716352,
  44907439,
  0,
  datetime.datetime(2019, 11, 13, 13, 30, 3),
  None],
 ['SoloRenektonOnly',
  1.348640976673499,
  11653376,
  0,
  datetime.datetime(2019, 11, 13, 13, 30, 3),
  None],
 ['スタンミ',
  1.1269794381225222,
  5913220,
  0,
  datetime.datetime(2019, 11, 13, 13, 30, 3),
  None],
 ['과로사1',
  0.9176871647486717,
  1894834,
  1,
  datetime.datetime(2019, 11, 13, 13, 30, 3),
  None],
 ['狂暴小建',
  0.7584563614489315,
  47640237,
  0,
 

In [83]:
df = pd.DataFrame(data, columns = ['UserName', 'DeltaPercentage',"TotalViewerCount","LeaversFollowers","RequestTime","Distance"]) 
df

,UserName,DeltaPercentage,TotalViewerCount,LeaversFollowers,RequestTime,Distance
0,한동숙,8.920359,50998396,1,2019-11-13 13:30:02,1.0
1,超負荷,1.926891,10773880,0,2019-11-13 13:30:02,2.0
2,接接,2.840147,103055955,0,2019-11-13 13:30:03,2.0
3,NoWay4u_Sir,3.008032,38866827,0,2019-11-13 13:30:03,2.0
4,Solary,1.087689,87110241,0,2019-11-13 13:30:03,NaN
5,Rakin,1.865333,44907439,0,2019-11-13 13:30:03,1.0
6,SoloRenektonOnly,1.348641,11653376,0,2019-11-13 13:30:03,2.0
7,スタンミ,1.126979,5913220,0,2019-11-13 13:30:03,NaN
8,과로사1,0.917687,1894834,1,2019-11-13 13:30:03,1.0
9,狂暴小建,0.758456,47640237,0,2019-11-13 13:30:04,1.0


In [8]:
print(streams1)
print("#############################")
print(streams2)

([('Faker', 34836, 43691, datetime.datetime(2019, 11, 13, 13, 0, 3)), ('한동숙', 19594, 139470326, datetime.datetime(2019, 11, 13, 13, 0, 3)), ('超負荷', 6180, 29506118, datetime.datetime(2019, 11, 13, 13, 0, 3)), ('괴물쥐123', 5883, 148057505, datetime.datetime(2019, 11, 13, 13, 0, 3)), ('Solary', 5647, 174955366, datetime.datetime(2019, 11, 13, 13, 0, 4)), ('接接', 4370, 11561802, datetime.datetime(2019, 11, 13, 13, 0, 4)), ('Rakin', 4164, 44099416, datetime.datetime(2019, 11, 13, 13, 0, 4)), ('RATIRL', 3907, 57292293, datetime.datetime(2019, 11, 13, 13, 0, 4)), ('NoWay4u_Sir', 3648, 85397463, datetime.datetime(2019, 11, 13, 13, 0, 4)), ('테디이', 3540, 152714599, datetime.datetime(2019, 11, 13, 13, 0, 4)), ('과로사1', 2110, 439360392, datetime.datetime(2019, 11, 13, 13, 0, 4)), ('スタンミ', 2109, 37323664, datetime.datetime(2019, 11, 13, 13, 0, 5)), ('狂暴小建', 2084, 25236843, datetime.datetime(2019, 11, 13, 13, 0, 5)), ('GoBGG', 1956, 40760591, datetime.datetime(2019, 11, 13, 13, 0, 5)), ('SoloRenektonOnl

In [9]:

time1 = datetime(2019, 11, 15, 17, 0) # 2019-11-11 17:00:02
time2 = datetime(2019, 11, 15, 18,0) # 2019-11-16 04:45:07

def get_progression(start_date,end_date):
    DBase=[]
    data = start_date
    safedata=start_date+timedelta(minutes=10)
    while(data < end_date):
        stream1=list_streamer_time(data,safedata,game)
        streams2=list_streamer_time(data+timedelta(minutes=30),safedata+timedelta(minutes=30),game)
        DBase+=generate_dataframe(streams1,streams2)
        data += timedelta(minutes=30)
        safedata+=timedelta(minutes=30)
    df = pd.DataFrame(DBase, columns = ['UserName', 'DeltaPercentage',"TotalViewerCount","LeaversFollowers","RequestTime"])
    print("Acaaabou!")
    return df

multipla = get_progression(time1,time2)

SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU
SAIU


OperationalError: (2013, 'Lost connection to MySQL server during query ([Errno 54] Connection reset by peer)')

In [ ]:
print(multipla)
multipla.to_csv("total.csv")

In [ ]:
fm.linregress(multipla, ['TotalViewerCount', 'LeaversFollowers'], 'DeltaPercentage')

In [ ]:
print(coefs)